In [32]:
import pandas as pd
import numpy as np
import plotly.express as px
import os
import sys
from rdp import rdp
from sklearn.cluster import DBSCAN


ModuleNotFoundError: No module named 'sklearn'

### Data Import
If you need to create a unified dataset run data below.
function;
The below function will go through each data file in Geolife Trajectories 1.3 file. Function label each import with the user it came from. It then save the new data to the file "totalDataSetLocaiton.csv".


In [26]:
fileLocation = "./data/Geolife Trajectories 1.3/Data"
totalDataset = pd.DataFrame()
alluserInfo = []
for folder in os.listdir(fileLocation):
    for day in os.listdir(os.path.join(fileLocation,folder,"Trajectory/")):
        user = pd.read_csv(os.path.join(fileLocation,folder,"Trajectory/",day), skiprows=6, names= ["lat","long","zero","altitude","timeValue","date", "time"], dtype={"lat":np.float64,"long":np.float64,"altitude":np.float64,"timeValue":np.float64})
        user['userNumber'] = pd.Series([folder for x in range(len(user.index))])
        alluserInfo.append(user)

totalDataset = pd.concat(alluserInfo)
totalDataset.head()

,lat,long,zero,altitude,timeValue,date,time,userNumber
0,39.974967,116.329317,0,157.480315,39573.596366,2008-05-05,14:18:46,053
1,39.975250,116.338183,0,141.076115,39573.597708,2008-05-05,14:20:42,053
2,39.975417,116.339017,0,236.220472,39573.598137,2008-05-05,14:21:19,053
3,39.975250,116.342700,0,239.501312,39573.598634,2008-05-05,14:22:02,053
4,39.975333,116.344917,0,239.501312,39573.598854,2008-05-05,14:22:21,053


In [27]:
totalDataset["datetime"] = pd.to_datetime(totalDataset['date'] + ' ' + totalDataset['time'])
totalDataset["timestamp"] = totalDataset['datetime'].apply(lambda x: x.timestamp())
totalDataset["latlongtime"] = totalDataset[['lat','long','timestamp']].values.tolist()

In [28]:
totalDataset.to_csv("./data/totalDataSetLocation.csv")

### Find Location
Plan;
I know data is collected at a frequency of 1-5 seconds or ever 5~10 meters. So if a location is report for more than 3 minutes within 10 meters we can assume the user stoped her for a prolong period of time. I would expect at least 3 different poi per day. Of course this is recorded before covid so we assume user will not be working from home so they have more points of interest. I will aim for at least 3 different poi per user per day using this standard.

Assumptions:
A possible point of interest is define as a user stopping in a 10 meter radius for more then 3 minues. We will see how many poi frequency.

Plan1 - so we will use ramer-douglas-peucker to minimize the amount of points. This will then be will then minimize the total amountof points. allow me to the calculate time at each point.
plan2 - clustering the points

- So I really want to see first if i can

In [29]:
totalDataset.head()

,lat,long,zero,altitude,timeValue,date,time,userNumber,datetime,timestamp,latlongtime
0,39.974967,116.329317,0,157.480315,39573.596366,2008-05-05,14:18:46,053,2008-05-05 14:18:46,1.209997e+09,"[39.9749666666667, 116.329316666667, 120999712..."
1,39.975250,116.338183,0,141.076115,39573.597708,2008-05-05,14:20:42,053,2008-05-05 14:20:42,1.209997e+09,"[39.97525, 116.338183333333, 1209997242.0]"
2,39.975417,116.339017,0,236.220472,39573.598137,2008-05-05,14:21:19,053,2008-05-05 14:21:19,1.209997e+09,"[39.9754166666667, 116.339016666667, 120999727..."
3,39.975250,116.342700,0,239.501312,39573.598634,2008-05-05,14:22:02,053,2008-05-05 14:22:02,1.209997e+09,"[39.97525, 116.3427, 1209997322.0]"
4,39.975333,116.344917,0,239.501312,39573.598854,2008-05-05,14:22:21,053,2008-05-05 14:22:21,1.209997e+09,"[39.9753333333333, 116.344916666667, 120999734..."


In [ ]:
limiteduserInfo = []

for name, uday in totalDataset.groupby(["userNumber","date"]):
    userDay = pd.DataFrame(rdp(list(uday.latlongtime), epsilon=0.001), columns=["lat","long","time"])
    userDay['userNumber'] = pd.Series([name[0] for x in range(len(userDay.index))])
    limiteduserInfo.append(userDay)

limtedDataSet = pd.concat(limiteduserInfo)


In [ ]:
limtedDataSet.describe()

So i have a smaller dataset now to user the DBSCAN clustering alorithm on.

min_samples = 2 - assuming people odnt spend hte whole day at home so should have at least 2 poin of interest on any given day. This should give enough POI to work with